In [1]:
import numpy as np
import import_ipynb
from dumbAgent import DumbAgent
from environment import TicTacToe3D
import pygame, sys

importing Jupyter notebook from dumbAgent.ipynb
importing Jupyter notebook from environment.ipynb
pygame 2.5.2 (SDL 2.28.2, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# dummy_board = np.zeros((4, 4, 4), dtype=int)
# # floor, row, col
# dummy_board[0, 0, 0] = 1
# dummy_board[0, 1, 1] = 1
# dummy_board[0, 2, 2] = 1

# agent = DumbAgent()
# x = agent.findBestMove(dummy_board, 1)
# print(x)

In [3]:
agent = DumbAgent()

env = TicTacToe3D(headless=False)
env.draw_lines()
pygame.display.update()

player = 1

while True:
	command = input()
	
	if command == 'q':
		pygame.quit()
		break
	else:
		x, y = [int(a) for a in command.split(" ")]

		if not env.move(x, y, player):
			print('Invalid move')
			continue
		
		print(f"PLAYER: {x}, {y}")
		env.draw_figures()
		player = -player
		pygame.display.update()
    
		if (result := env.check()):
			print('Player', 'OAB'[result], 'wins!')
			break

        # For BOT
		botX, botyY = agent.findBestMove(env.board, player)
		print(f"BOT: {botX}, {botyY}")
		env.move(botX, botyY, player)
		env.draw_figures()
		player = -player
		pygame.display.update()
    
		if (result := env.check()):
			print('Player', 'OAB'[result], 'wins!')
			break

PLAYER: 0, 0
BOT: 0, 3
PLAYER: 1, 0
BOT: 3, 0
PLAYER: 1, 2
BOT: 1, 1
PLAYER: 0, 2
BOT: 1, 1
PLAYER: 2, 2
BOT: 3, 2
PLAYER: 1, 2
BOT: 0, 0
PLAYER: 3, 2
BOT: 0, 0
PLAYER: 0, 2
BOT: 0, 0
PLAYER: 2, 2
Player A wins!


: 